In [5]:
# 这里先演示一下最简单的代理使用方法
import requests

proxy = 'http://127.0.0.1:7890'

proxies = {
    'http':  proxy,
    'https': proxy,
}
# 我们使用代理访问谷歌
try:
    response = requests.get('https://www.google.com', proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print('Error', e.args)

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="zh-TW"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="MKywd87CGQOJlLqjDHU0Sg">(function(){window.google={kEI:'wZeiYuG7Munt2roPq4au-AI',kEXPI:'0,1302536,56873,6058,207,2414,2390,896,1420,383,246,5,1354,4013,1123753,1197700,380791,16114,28684,17572,4859,1361,9291,3026,4748,12834,4998,13227,3848,10622,7432,15309,5081,889,704,1279,2742,149,1103,840,1983,4314,108,3406,606,2023,1777,520,14670,3227,2845,7,17450,16320,1851,15756,3,576,6459,149,13975,4,1528,2304,32112,2658,7356,13659,4437,16786,652,5163,2542,4094,17,4035,3,3541,1,42154,2,14022,2558,11558,11623,5679,1021,2380,2718,18243,2,6,7772,4569,6258,23418,830,422,4570,2,6,1257,12141,2826,4333,20,4996,1074,1394,445,2,2,1,10790,15842,8155,6582,799,3043,11637,2322,5019,3421,2,6,11027,1928,5609,97,916,2547,58,552,2

In [ ]:
# 如果代理有用户名密码，可以这样
# user:password@127.0.0.1:7890

In [6]:
# pip3 install PySocks

# 如果是socket代理可以这样
import socks
import socket
from urllib import request
from urllib.error import URLError

# 这里我们使用socks5类型
socks.set_default_proxy(socks.SOCKS5, '127.0.0.1', 7890)
socket.socket = socks.socksocket
try:
    response = request.urlopen('http://httpbin.org/get')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7", 
    "X-Amzn-Trace-Id": "Root=1-62a29f5c-7817985a588ee2996eb16292"
  }, 
  "origin": "111.252.215.46", 
  "url": "http://httpbin.org/get"
}



In [7]:
# requests模块的sockes5代理可以这样使用
proxy = '127.0.0.1:7890'
proxies = {
    'http': 'socks5://' + proxy,
    'https': 'socks5://' + proxy
}
try:
    response = requests.get('http://httpbin.org/get', proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print('Error', e.args)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.27.1", 
    "X-Amzn-Trace-Id": "Root=1-62a2a268-3aadb3797ac449b33209101d"
  }, 
  "origin": "111.252.215.46", 
  "url": "http://httpbin.org/get"
}



In [8]:
# selenium 也可以设置代理
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=http://127.0.0.1:7890')
browser = webdriver.Chrome(chrome_options=chrome_options)
browser.get('https://www.google.com')

d:\app\python\377\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
# 如果是认证代理，那么使用起来就会比较麻烦，这里就直接贴一下代码
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import zipfile

ip = '127.0.0.1'
port = 9743
username = 'foo'
password = 'bar'

manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    }
}
"""

background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
          singleProxy: {
            scheme: "http",
            host: "%(ip)s",
            port: %(port)s
          }
        }
      }

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%(username)s",
            password: "%(password)s"
        }
    }
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
)
""" % {'ip': ip, 'port': port, 'username': username, 'password': password}

plugin_file = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(plugin_file, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_extension(plugin_file)
browser = webdriver.Chrome(chrome_options=chrome_options)
browser.get('http://httpbin.org/get')